In [4]:
import pandas as pd

# Load training files
train_data = pd.read_csv(r"C:\Users\91739\OneDrive\Desktop\project tamucc\train.csv")
train_labels = pd.read_csv(r"C:\Users\91739\OneDrive\Desktop\project tamucc\label_train.csv")

# Load validation files
validate_data = pd.read_csv(r"C:\Users\91739\OneDrive\Desktop\project tamucc\validate.csv")
validate_labels = pd.read_csv(r"C:\Users\91739\OneDrive\Desktop\project tamucc\label_validate.csv")


In [5]:
import numpy as np

# Define the number of features
num_features = 1433

# Create a random solution (chromosome)
random_solution = np.random.randint(2, size=num_features)

# Display the random solution
print(random_solution)
print(type(random_solution))
print(len(random_solution))
#print()

[1 1 1 ... 1 0 1]
<class 'numpy.ndarray'>
1433


In [6]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Assuming you already have the population of solutions
population_size = 108
num_features = 1433
population = np.random.randint(2, size=(population_size, num_features))

# Split the validation set into features and labels
X_validate = validate_data.values  # Assuming validate_data is your validation feature data
y_validate = validate_labels.values.flatten()  # Assuming validate_labels is your validation label data

# Initialize a list to store the fitness scores
fitness_scores = []

# Evaluate fitness for each solution
for solution in population:
    # Select features based on the solution
    selected_features = X_validate[:, solution == 1]

    # Split the features and labels into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(selected_features, y_validate, test_size=0.2, random_state=42)

    # Train a model (example: Logistic Regression)
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Evaluate accuracy on the validation set
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    # Append the accuracy to the fitness scores list
    fitness_scores.append(accuracy)

# Sort solutions based on fitness scores (descending order)
sorted_indices = np.argsort(fitness_scores)[::-1]
sorted_population = population[sorted_indices]

# Display the sorted population
print("Sorted population based on accuracy:")
print(sorted_population)


Sorted population based on accuracy:
[[1 0 1 ... 1 0 0]
 [0 1 1 ... 1 0 1]
 [1 0 1 ... 0 1 0]
 ...
 [0 0 0 ... 1 0 1]
 [1 0 1 ... 0 1 1]
 [1 1 1 ... 1 0 1]]


In [18]:
import numpy as np

def roulette_wheel_selection(population, fitness_scores):
    # Normalize fitness scores
    normalized_fitness = fitness_scores / np.sum(fitness_scores)
    
    # Compute cumulative probabilities
    cumulative_probs = np.cumsum(normalized_fitness)
    
    # Select solutions probabilistically
    selected_indices = []
    for _ in range(len(population)):
        r = np.random.rand()  # Random number between 0 and 1
        for i, cum_prob in enumerate(cumulative_probs):
            if r <= cum_prob:
                selected_indices.append(i)
                break
                
    return population[selected_indices]

# Example usage
# population = np.random.randint(2, size=(100, num_features))  # Example population
# fitness_scores = np.random.rand(100)  # Example fitness scores

# Initialize the population
population = np.random.randint(2, size=(population_size, num_features))  # Ensure num_features matches the actual number of features

# Main loop for the genetic algorithm
for generation in range(num_generations):
    # Evaluate fitness of the population
    fitness_scores = evaluate_fitness(population, X_train, y_train, X_val, y_val)  # Pass X_train, y_train, X_val, y_val to evaluate_fitness

    # Rest of the genetic algorithm implementation...

selected_solutions = roulette_wheel_selection(population, fitness_scores)


IndexError: boolean index did not match indexed array along dimension 1; dimension is 752 but corresponding boolean dimension is 1433

In [8]:
def single_point_crossover(parent1, parent2):
    # Select a random crossover point
    crossover_point = np.random.randint(1, len(parent1))

    # Perform crossover
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

    return child1, child2

# Example usage
parent1 = np.array([1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1])
parent2 = np.array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0])

child1, child2 = single_point_crossover(parent1, parent2)
print("Parent 1:", parent1)
print("Parent 2:", parent2)
print("Child 1:", child1)
print("Child 2:", child2)


Parent 1: [1 0 1 0 1 0 1 1 0 0 1]
Parent 2: [0 1 0 1 0 1 0 0 1 1 0]
Child 1: [1 0 1 1 0 1 0 0 1 1 0]
Child 2: [0 1 0 0 1 0 1 1 0 0 1]


In [9]:
def mutation(chromosome, mutation_rate):
    mutated_chromosome = chromosome.copy()
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            # Flip the bit (gene) with probability mutation_rate
            mutated_chromosome[i] = 1 - mutated_chromosome[i]
    return mutated_chromosome

# Example usage
offspring = np.array([1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1])  # Example offspring chromosome
mutation_rate = 0.1  # Mutation rate (probability of mutation for each gene)

mutated_offspring = mutation(offspring, mutation_rate)
print("Offspring before mutation:", offspring)
print("Mutated offspring:", mutated_offspring)


Offspring before mutation: [1 0 1 0 1 0 1 1 0 0 1]
Mutated offspring: [1 0 1 0 1 0 1 1 0 1 1]


In [10]:
def replace_population(old_population, offspring, num_elites):
    # Sort the old population based on fitness (if necessary)
    # ...

    # Select the top 'num_elites' best-performing parent chromosomes
    elites = old_population[:num_elites]

    # Combine elites and offspring to form the new population
    new_population = np.concatenate((elites, offspring))

    return new_population

# Example usage
old_population = np.random.randint(2, size=(100, num_features))  # Example old population
offspring = np.random.randint(2, size=(50, num_features))  # Example offspring chromosomes
num_elites = 10  # Number of best-performing parent chromosomes to retain in the new population

new_population = replace_population(old_population, offspring, num_elites)


In [17]:
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# def evaluate_fitness(population):
#     fitness_scores = []
#     for individual in population:
#         # Select features based on the individual's chromosome
#         selected_features = X_train[:, individual == 1]
        

#         # Train a support vector machine classifier
#         model = SVC()
#         model.fit(selected_features, y_train)

#         # Evaluate accuracy on the validation set
#         selected_features_val = X_val[:, individual == 1]
#         y_pred = model.predict(selected_features_val)
#         accuracy = accuracy_score(y_val, y_pred)

#         # Append accuracy to the fitness scores list
#         fitness_scores.append(accuracy)

#     return fitness_scores
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def evaluate_fitness(population, X_train, y_train, X_val, y_val):
    fitness_scores = []
    for individual in population:
        # Select features based on the individual's chromosome
        selected_features = X_train[:, individual == 1]

        # Train a support vector machine classifier
        model = SVC()
        model.fit(selected_features, y_train)

        # Evaluate accuracy on the validation set
        selected_features_val = X_val[:, individual == 1]
        y_pred = model.predict(selected_features_val)
        accuracy = accuracy_score(y_val, y_pred)

        # Append accuracy to the fitness scores list
        fitness_scores.append(accuracy)

    return fitness_scores


In [14]:
print("X_train shape:", X_train.shape)
print("Boolean index shape:", individual.shape)


X_train shape: (85, 752)


NameError: name 'individual' is not defined

In [13]:
def genetic_algorithm(population_size, num_features, num_generations, crossover_rate, mutation_rate, num_elites):
    # Initialize the population
    population = np.random.randint(2, size=(population_size, num_features))

    # Main loop for the genetic algorithm
    for generation in range(num_generations):
        # Evaluate fitness of the population
        fitness_scores = evaluate_fitness(population)  # Implement your fitness evaluation function

        # Select parent chromosomes for crossover
        selected_parents = roulette_wheel_selection(population, fitness_scores)

        # Apply crossover to create offspring
        offspring = []
        for i in range(0, population_size, 2):
            if np.random.rand() < crossover_rate:
                parent1 = selected_parents[i]
                parent2 = selected_parents[i+1]
                child1, child2 = single_point_crossover(parent1, parent2)
                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.append(selected_parents[i])
                offspring.append(selected_parents[i+1])

        # Apply mutation to offspring
        for i in range(len(offspring)):
            offspring[i] = mutation(offspring[i], mutation_rate)

        # Replace the old population with the new population
        population = replace_population(population, offspring, num_elites)

        # Termination condition (example: stop after a fixed number of generations)
        if termination_condition_met():
            break

    return population

# Example usage
population_size = 100
num_features = 1433
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.01
num_elites = 10

final_population = genetic_algorithm(population_size, num_features, num_generations, crossover_rate, mutation_rate, num_elites)


IndexError: boolean index did not match indexed array along dimension 1; dimension is 752 but corresponding boolean dimension is 1433